In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print (os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_4.bin', 'data_batch_2.bin', 'test_batch.bin', 'readme.html', 'data_batch_5.bin', 'data_batch_1.bin', 'data_batch_3.bin']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open(filename,'rb') as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'],data[b'labels']
    
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_lebels = []
        for filename in filenames:
            data,labels = load_data(filename)
            all_data.append(data)
            all_lebels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1
        self._labels = np.hstack(all_lebels)
        '''print (self._data.shape)
        print (self._labels.shape)
        '''
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        #[0,1,2,3,4,5]-> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels
                
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d.bin' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch.bin')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,False)

MemoryError: 

In [ ]:
x = tf.placeholder(tf.float32,[None,3072])
#[None]
y = tf.placeholder(tf.int64,[None])

# (3072,10)
w = tf.get_variable('w',[x.get_shape()[-1], 10],
                    initializer = tf.random_normal_initializer(0, 1))
#(10, )
b = tf.get_variable('b',[10],
                    initializer = tf.constant_initializer(0.0))
# [None,3072]*[3072,10] = [None,10]
y_ = tf.matmul(x,w) + b
"""
# 
#
#
p_y = tf.nn.softmax(y_)
#5 -> [0,0,0,0,0,1,0,0,0,0]
y_one_hot = tf.one_hot(y,10,dtype=tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
"""

loss = tf.losses.sparse_softm ax_cross_entropy(labels=y, logits=y_)
#y_ -> sofmax
#y -> one_hot
#loss = ylogy_
#bool
predict = tf.argmax(y_,1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict,y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [ ]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val , acc_val,_ = sess.run(
            [loss,accuracy,train_op],
            feed_dict={
                x: batch_data,y: batch_labels})
        """
        for i in range(0, len(mnist.test.images)):
  result = sess.run(correct_prediction, feed_dict={x: np.array([mnist.test.images[i]]), y_: np.array([mnist.test.labels[i]])})
  if not result:
    print('预测的值是：',sess.run(y, feed_dict={x: np.array([mnist.test.images[i]]), y_: np.array([mnist.test.labels[i]])}))
    print('实际的值是：',sess.run(y_,feed_dict={x: np.array([mnist.test.images[i]]), y_: np.array([mnist.test.labels[i]])}))
    one_pic_arr = np.reshape(mnist.test.images[i], (28, 28))
    pic_matrix = np.matrix(one_pic_arr, dtype="float")
    plt.imshow(pic_matrix)
    pylab.show()
    break
 
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                     y_: mnist.test.labels})) """ 
        if (i+1) % 500 == 0:
            print('[Train] Step: %d,loss: %4.5f,acc: %4.5f'  \
                  % (i+1,loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames,False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels  = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],feed_dict = {x: test_batch_data,y: test_batch_labels})
                
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' \
                  % (i+1 ,test_acc))